(**You can also open this notebook in Google Colab**)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiangshiyin/data-programming-with-python/blob/main/2023-fall/2023-10-03/notebook/code_demo.ipynb)

# Question from last class

## Set data type of columsn with missing value

### The problem

In [1]:
import pandas as pd
import numpy as np

tb1 = pd.DataFrame({'key': ['foo', 'boo', 'foo'], 'lval': [1, 2, 3]})
tb2 = pd.DataFrame({'key': ['foo', 'coo'], 'rval': [5, 6]})

In [2]:
tb1

,key,lval
0,foo,1
1,boo,2
2,foo,3


In [3]:
tb2

,key,rval
0,foo,5
1,coo,6


In [4]:
outer_join = pd.merge(tb1, tb2, on='key', how='outer')
outer_join

,key,lval,rval
0,foo,1.0,5.0
1,foo,3.0,5.0
2,boo,2.0,NaN
3,coo,NaN,6.0


In [5]:
outer_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key     4 non-null      object 
 1   lval    3 non-null      float64
 2   rval    3 non-null      float64
dtypes: float64(2), object(1)
memory usage: 224.0+ bytes


In [6]:
pd.__version__

'2.1.1'

In [7]:
outer_join.lval.astype('int')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

### Workarounds

Reference: 
- `df.astype()` function in detail: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html
- Stackoverflow: https://stackoverflow.com/questions/48511484/data-type-conversion-error-valueerror-cannot-convert-non-finite-values-na-or
- Nullable integer type: https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html#integer-na

In [8]:
outer_join.lval.fillna(-1).astype(int)

0    1
1    3
2    2
3   -1
Name: lval, dtype: int64

In [9]:
outer_join.lval.astype('Int64')

0       1
1       3
2       2
3    <NA>
Name: lval, dtype: Int64

In [10]:
outer_join.lval.astype(np.int64)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

* Instead of asking "what could happen", try and break the code
* Research online and read documentations

# `dataframe` practice (continued)

In [12]:
import pandas as pd
import numpy as np

In [13]:
Employees = pd.read_excel('../data/Employees.xls')
Territory = pd.read_excel('../data/SalesTerritory.xls')
Customers = pd.read_excel('../data/Customers.xls')
Orders = pd.read_excel('../data/ItemsOrdered.xls')

## Filter
### Change the logic to include anyone who meets any of the 3 conditions (i.e., people who are either married, live in Washington state, or have more than 35 vacation hours left)

SQL logic
```sql
SELECT 	
  e.EmployeeID
  ,e.FirstName
  ,e.LastName
  ,e.MaritalStatus
  ,e.VacationHours
  ,e.SalariedFlag
  ,e.StateProvinceName
  ,e.CountryName
FROM dbo.Employees AS e
WHERE 
  e.MaritalStatus = 'M' 
  OR e.VacationHours > 35 
  OR e.StateProvinceName = 'Washington'
	;
```

In [14]:
Employees.loc[(Employees.MaritalStatus=='M')|(Employees.VacationHours>35)|(Employees.StateProvinceName=='Washington'), 
              ['EmployeeID', 'FirstName', 'LastName','MaritalStatus','VacationHours','SalariedFlag','StateProvinceName','CountryName']].head(3)

,EmployeeID,FirstName,LastName,MaritalStatus,VacationHours,SalariedFlag,StateProvinceName,CountryName
0,259,Ben,Miller,M,55,0,Washington,United States
1,278,Garrett,Vargas,M,33,1,Alberta,Canada
2,204,Gabe,Mares,M,57,0,Washington,United States


In [15]:
Employees.loc[(Employees.MaritalStatus=='M')|(Employees.VacationHours>35)|(Employees.StateProvinceName=='Washington'), 
              ['EmployeeID', 'FirstName', 'LastName','MaritalStatus','VacationHours','SalariedFlag','StateProvinceName','CountryName']].EmployeeID.nunique()

286

## Joins
![](../pics/joins.jpg)

### If any are salespeople then show me the details about their sales territory
```sql
SELECT e.EmployeeID ,e.FirstName + ' ' + e.LastName AS EmployeeName ,st.* 
FROM dbo.Employees AS e 
INNER JOIN dbo.SalesTerritory AS st ON e.TerritoryID = st.TerritoryID
```

In [16]:
Territory.shape

(12, 6)

In [17]:
Territory

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear
0,1,Northwest,US,North America,7887186.79,3298694.490
1,2,Northeast,US,North America,2402176.85,3607148.940
2,3,Central,US,North America,3072175.12,3205014.080
3,4,Southwest,US,North America,10510853.87,5366575.710
4,5,Southeast,US,North America,2538667.25,3925071.430
5,6,Canada,CA,North America,6771829.14,5693988.860
6,7,France,FR,Europe,4772398.31,2396539.760
7,8,Germany,DE,Europe,3805202.35,1307949.790
8,9,Australia,AU,Pacific,5977814.92,2278548.980
9,10,United Kingdom,GB,Europe,5012905.37,1635823.400


In [18]:
Employees.columns

Index(['EmployeeID', 'ManagerID', 'TerritoryID', 'Title', 'FirstName',
       'MiddleName', 'LastName', 'Suffix', 'JobTitle', 'NationalIDNumber',
       'BirthDate', 'MaritalStatus', 'Gender', 'HireDate', 'SalariedFlag',
       'VacationHours', 'SickLeaveHours', 'PhoneNumber', 'PhoneNumberType',
       'EmailAddress', 'AddressLine1', 'AddressLine2', 'City',
       'StateProvinceName', 'PostalCode', 'CountryName'],
      dtype='object')

In [20]:
Employees.head(3)

,EmployeeID,ManagerID,TerritoryID,Title,FirstName,MiddleName,LastName,Suffix,JobTitle,NationalIDNumber,...,SickLeaveHours,PhoneNumber,PhoneNumberType,EmailAddress,AddressLine1,AddressLine2,City,StateProvinceName,PostalCode,CountryName
0,259,250.0,NaN,NaN,Ben,T,Miller,NaN,Buyer,20269531,...,47,151-555-0113,Work,ben0@adventure-works.com,101 Candy Rd.,NaN,Redmond,Washington,98052,United States
1,278,274.0,6.0,NaN,Garrett,R,Vargas,NaN,Sales Representative,234474252,...,36,922-555-0165,Work,garrett1@mapleleafmail.ca,10203 Acorn Avenue,NaN,Calgary,Alberta,T2P 2G8,Canada
2,204,26.0,NaN,NaN,Gabe,B,Mares,NaN,Production Technician - WC40,440379437,...,48,310-555-0117,Work,gabe0@adventure-works.com,1061 Buskrik Avenue,NaN,Edmonds,Washington,98020,United States


In [21]:
df = pd.merge(
    Employees.loc[:,['TerritoryID','EmployeeID','FirstName','LastName']],
    Territory,
    on = 'TerritoryID',
    how = 'inner'
)
df.head(3)

,TerritoryID,EmployeeID,FirstName,LastName,Name,CountryCode,Region,SalesYTD,SalesLastYear
0,6.0,278,Garrett,Vargas,Canada,CA,North America,6771829.14,5693988.86
1,6.0,282,José,Saraiva,Canada,CA,North America,6771829.14,5693988.86
2,1.0,283,David,Campbell,Northwest,US,North America,7887186.79,3298694.49


In [22]:
df.EmployeeID.nunique()

14

In [23]:
df = pd.merge(
    Employees.loc[:,['TerritoryID','EmployeeID','FirstName','LastName']],
    Territory,
    on = 'TerritoryID',
    how = 'left'
)
df.head(3)

,TerritoryID,EmployeeID,FirstName,LastName,Name,CountryCode,Region,SalesYTD,SalesLastYear
0,NaN,259,Ben,Miller,NaN,NaN,NaN,NaN,NaN
1,6.0,278,Garrett,Vargas,Canada,CA,North America,6771829.14,5693988.86
2,NaN,204,Gabe,Mares,NaN,NaN,NaN,NaN,NaN


In [24]:
df.shape

(291, 9)

In [25]:
Employees.shape

(291, 26)

In [26]:
df['EmployeeName'] = [
    first + ' ' + last
    for first,last in zip(df.FirstName, df.LastName)
]

In [27]:
df.head(3)

,TerritoryID,EmployeeID,FirstName,LastName,Name,CountryCode,Region,SalesYTD,SalesLastYear,EmployeeName
0,NaN,259,Ben,Miller,NaN,NaN,NaN,NaN,NaN,Ben Miller
1,6.0,278,Garrett,Vargas,Canada,CA,North America,6771829.14,5693988.86,Garrett Vargas
2,NaN,204,Gabe,Mares,NaN,NaN,NaN,NaN,NaN,Gabe Mares


### For all sales territories, also show what customers fall under them
```sql
SELECT * 
FROM dbo.SalesTerritory AS st 
LEFT OUTER JOIN dbo.Customers AS c ON c.SalesTerritoryID = st.TerritoryID ;
```

In [28]:
Customers.head(3)

,CustomerID,SalesTerritoryID,FirstName,LastName,City,StateName
0,10101,1,John,Gray,Lynden,Washington
1,10298,4,Leroy,Brown,Pinetop,Arizona
2,10299,1,Elroy,Keller,Snoqualmie,Washington


In [29]:
Territory.head(3)

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear
0,1,Northwest,US,North America,7887186.79,3298694.49
1,2,Northeast,US,North America,2402176.85,3607148.94
2,3,Central,US,North America,3072175.12,3205014.08


In [30]:
Territory.shape

(12, 6)

In [31]:
df = pd.merge(
    Territory,
    Customers,
#     on = 'TerritoryID',
    left_on='TerritoryID',
    right_on='SalesTerritoryID',
    how = 'left'
)
df.head(3)

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear,CustomerID,SalesTerritoryID,FirstName,LastName,City,StateName
0,1,Northwest,US,North America,7887186.79,3298694.49,10101.0,1.0,John,Gray,Lynden,Washington
1,1,Northwest,US,North America,7887186.79,3298694.49,10299.0,1.0,Elroy,Keller,Snoqualmie,Washington
2,1,Northwest,US,North America,7887186.79,3298694.49,10325.0,1.0,Ginger,Schultz,Pocatello,Idaho


In [32]:
df.shape

(25, 12)

In [33]:
df = pd.merge(
    Territory,
    Customers.rename(columns={'SalesTerritoryID':'TerritoryID'}),
    on = 'TerritoryID',
    how = 'left'
)
df.head(3)

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear,CustomerID,FirstName,LastName,City,StateName
0,1,Northwest,US,North America,7887186.79,3298694.49,10101.0,John,Gray,Lynden,Washington
1,1,Northwest,US,North America,7887186.79,3298694.49,10299.0,Elroy,Keller,Snoqualmie,Washington
2,1,Northwest,US,North America,7887186.79,3298694.49,10325.0,Ginger,Schultz,Pocatello,Idaho


Are there any sales territories that don't have any customers associated?

In [34]:
# df.loc[condition, column_list]
df[df.CustomerID.isna()]

,TerritoryID,Name,CountryCode,Region,SalesYTD,SalesLastYear,CustomerID,FirstName,LastName,City,StateName
5,2,Northeast,US,North America,2402176.85,3607148.940,NaN,NaN,NaN,NaN,NaN
18,6,Canada,CA,North America,6771829.14,5693988.860,NaN,NaN,NaN,NaN,NaN
19,7,France,FR,Europe,4772398.31,2396539.760,NaN,NaN,NaN,NaN,NaN
20,8,Germany,DE,Europe,3805202.35,1307949.790,NaN,NaN,NaN,NaN,NaN
21,9,Australia,AU,Pacific,5977814.92,2278548.980,NaN,NaN,NaN,NaN,NaN
22,10,United Kingdom,GB,Europe,5012905.37,1635823.400,NaN,NaN,NaN,NaN,NaN
23,11,Brazil,BR,South America,0.00,261589.958,NaN,NaN,NaN,NaN,NaN
24,12,Mexico,MX,North America,0.00,0.000,NaN,NaN,NaN,NaN,NaN


In [35]:
df[df.CustomerID.isna()].shape[0]

8

## Grouping

Reading Materials: 
* (official doc): https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html
* (summary) https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/

### What is the earliest birthdate for all employees?

SQL logic
```sql
SELECT MIN(e.BirthDate) FROM dbo.Employees AS e;
```

In [36]:
Employees.head(3)

,EmployeeID,ManagerID,TerritoryID,Title,FirstName,MiddleName,LastName,Suffix,JobTitle,NationalIDNumber,...,SickLeaveHours,PhoneNumber,PhoneNumberType,EmailAddress,AddressLine1,AddressLine2,City,StateProvinceName,PostalCode,CountryName
0,259,250.0,NaN,NaN,Ben,T,Miller,NaN,Buyer,20269531,...,47,151-555-0113,Work,ben0@adventure-works.com,101 Candy Rd.,NaN,Redmond,Washington,98052,United States
1,278,274.0,6.0,NaN,Garrett,R,Vargas,NaN,Sales Representative,234474252,...,36,922-555-0165,Work,garrett1@mapleleafmail.ca,10203 Acorn Avenue,NaN,Calgary,Alberta,T2P 2G8,Canada
2,204,26.0,NaN,NaN,Gabe,B,Mares,NaN,Production Technician - WC40,440379437,...,48,310-555-0117,Work,gabe0@adventure-works.com,1061 Buskrik Avenue,NaN,Edmonds,Washington,98020,United States


In [37]:
Employees.columns

Index(['EmployeeID', 'ManagerID', 'TerritoryID', 'Title', 'FirstName',
       'MiddleName', 'LastName', 'Suffix', 'JobTitle', 'NationalIDNumber',
       'BirthDate', 'MaritalStatus', 'Gender', 'HireDate', 'SalariedFlag',
       'VacationHours', 'SickLeaveHours', 'PhoneNumber', 'PhoneNumberType',
       'EmailAddress', 'AddressLine1', 'AddressLine2', 'City',
       'StateProvinceName', 'PostalCode', 'CountryName'],
      dtype='object')

In [38]:
type(Employees.dtypes)

pandas.core.series.Series

In [39]:
Employees.loc[:,['BirthDate']].head(3)

,BirthDate
0,1967-07-05
1,1969-03-07
2,1982-06-11


In [43]:
Employees.dtypes['BirthDate']

dtype('O')

In [44]:
str(Employees.dtypes['BirthDate'])

'object'

In [45]:
Employees.BirthDate.dtypes

dtype('O')

Reference:
- Lexicographic order: https://en.wikipedia.org/wiki/Lexicographic_order
- StackOverflow: https://stackoverflow.com/questions/45950646/what-is-lexicographical-order

In [46]:
'1970-01-01' < '2023-06-26'

True

In [47]:
Employees.BirthDate.min()

'1945-11-17'

In [48]:
Employees.BirthDate.max()

'1985-07-01'

In [49]:
Employees.BirthDate.nunique()

279

### Add to the above, the most recent birthdate for all employees

SQL logic
```sql
SELECT 
  MIN(e.BirthDate) AS 'Earliest Birthday'
  , MAX(e.BirthDate) AS 'Most Reecent Birthday'
FROM dbo.Employees AS e;
```

In [50]:
x = [4,5,1,2,3]
min(x), max(x)

(1, 5)

In [52]:
Employees.agg({'BirthDate':['min','max']}).T

# Employees.agg({'BirthDate':['min','max']})

,min,max
BirthDate,1945-11-17,1985-07-01


In [54]:
Employees.agg({'BirthDate':[min,max]}).T.reset_index(drop=True)

/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/2428413550.py:1: FutureWarning: The provided callable <built-in function min> is currently using Series.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  Employees.agg({'BirthDate':[min,max]}).T.reset_index(drop=True)
/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/2428413550.py:1: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  Employees.agg({'BirthDate':[min,max]}).T.reset_index(drop=True)


,min,max
0,1945-11-17,1985-07-01


### Show the above results broken down by gender

SQL logic
```sql
SELECT 
  e.Gender
  , MIN(e.BirthDate) AS 'Earliest Birthday'
  , MAX(e.BirthDate) AS 'Most Reecent Birthday'
FROM dbo.Employees AS e
GROUP BY e.Gender
;
```

In [55]:
Employees.groupby('Gender')['BirthDate'].min().reset_index()

,Gender,BirthDate
0,F,1946-10-29
1,M,1945-11-17


In [56]:
Employees.groupby('Gender').agg({'BirthDate':[min,max]})

/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/3113095537.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  Employees.groupby('Gender').agg({'BirthDate':[min,max]})
/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/3113095537.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  Employees.groupby('Gender').agg({'BirthDate':[min,max]})


BirthDate            
               min         max
Gender                        
F       1946-10-29  1985-07-01
M       1945-11-17  1985-05-07

In [57]:
Employees.groupby('Gender').agg(
    min_bday=('BirthDate',min),
    max_bday=('BirthDate',max)
).reset_index()

/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/513166010.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  Employees.groupby('Gender').agg(
/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/513166010.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  Employees.groupby('Gender').agg(


,Gender,min_bday,max_bday
0,F,1946-10-29,1985-07-01
1,M,1945-11-17,1985-05-07


### Show the above results broken down by gender, and salaried/hourly

SQL logic
```sql
SELECT 
  e.Gender
  , e.SalariedFlag
  , MIN(e.BirthDate) AS 'Earliest Birthday'
  , MAX(e.BirthDate) AS 'Most Reecent Birthday'
FROM dbo.Employees AS e
GROUP BY e.Gender, e.SalariedFlag
;
```

In [58]:
Employees.groupby(['Gender','SalariedFlag']).agg(
    min_bday=('BirthDate',min),
    max_bday=('BirthDate',max)
).reset_index()

/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/3511780075.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  Employees.groupby(['Gender','SalariedFlag']).agg(
/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/3511780075.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  Employees.groupby(['Gender','SalariedFlag']).agg(


,Gender,SalariedFlag,min_bday,max_bday
0,F,0,1948-05-25,1985-07-01
1,F,1,1946-10-29,1980-07-06
2,M,0,1946-04-03,1985-05-07
3,M,1,1945-11-17,1982-04-14


### What are the average vacation hours for all employees?

SQL logic
```sql
SELECT AVG(e.VacationHours)
FROM dbo.Employees AS e	
;
```

In [59]:
Employees.VacationHours.mean()

50.628865979381445

### Show the above results broken down and ordered by job title¶

SQL logic
```sql
SELECT 
  e.JobTitle
  , AVG(e.VacationHours) AS 'Average Vacation'
  , MIN(e.VacationHours) AS 'Minimum Vacation'
FROM dbo.Employees AS e
GROUP BY e.JobTitle
;
```

In [60]:
Employees.groupby('JobTitle')['VacationHours'].min().reset_index().head(3)

,JobTitle,VacationHours
0,Accountant,58
1,Accounts Manager,57
2,Accounts Payable Specialist,63


In [61]:
Employees.groupby('JobTitle')['VacationHours'].mean().reset_index().head(3)

,JobTitle,VacationHours
0,Accountant,58.5
1,Accounts Manager,57.0
2,Accounts Payable Specialist,63.5


In [62]:
Employees.groupby('JobTitle')['VacationHours'].apply(lambda x: sum(x)/len(x)).reset_index().head(3)

,JobTitle,VacationHours
0,Accountant,58.5
1,Accounts Manager,57.0
2,Accounts Payable Specialist,63.5


In [63]:
Employees.groupby('JobTitle').agg(
    avg_pto_left=('VacationHours',lambda x: sum(x)/len(x)),
    min_pto_left=('VacationHours',min)
).reset_index()

/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/2078384919.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  Employees.groupby('JobTitle').agg(


,JobTitle,avg_pto_left,min_pto_left
0,Accountant,58.5,58
1,Accounts Manager,57.0,57
2,Accounts Payable Specialist,63.5,63
3,Accounts Receivable Specialist,61.0,60
4,Application Specialist,72.5,71
...,...,...,...
63,Stocker,97.0,96
64,Tool Designer,8.5,8
65,Vice President of Engineering,1.0,1
66,Vice President of Production,64.0,64


In [66]:
output = Employees.groupby('JobTitle').agg(
    avg_pto_left=('VacationHours',lambda x: sum(x)/len(x)),
    min_pto_left=('VacationHours',min)
).reset_index()
output.sort_values(by=['avg_pto_left'], ascending=False).head(10)

/var/folders/78/njcscll93_s6cc27zw_h0pmr0000gn/T/ipykernel_16179/1626051807.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  output = Employees.groupby('JobTitle').agg(


,JobTitle,avg_pto_left,min_pto_left
9,Chief Executive Officer,99.0,99
63,Stocker,97.0,96
61,Shipping and Receiving Clerk,94.5,94
62,Shipping and Receiving Supervisor,93.0,93
25,Maintenance Supervisor,92.0,92
42,Production Technician - WC10,91.0,83
24,Janitor,89.5,88
18,Facilities Administrative Assistant,87.0,87
19,Facilities Manager,86.0,86
53,Quality Assurance Technician,83.5,82
